# NPGO CESM-LENS Definition

Date: October 9th, 2017

Given the experimental setup of the CESM LENS, it makes more sense to generate residuals by simply removing the ensemble mean. Manu Di Lorenzio uses this in his 2016 paper in Nature Climate Change with Nate Mantua (as well as in an upcoming GRL paper). His strategy is as follows (with some additional steps added from me):

** Use SST **

1. Remove ensemble mean from raw time series of each ensemble member (e.g. using NCOs).

2. Regrid the residuals to a 1$^{o}$x1$^{o}$ (180x360) standard global map.

3. Take the Northeast Pacific domain (180-110W and 25-62N)

4. Compute EOFs of the JFM (winter) seasonal means of SST_JFM(x, y, t), where t is now a JFM average of each year. Make sure to weight by the sqrt of the cosine of latitude.

5. The first EOF will be the PDO pattern and the 2nd EOF will be NPGO dynamics.

6. At this point, use the patterns of EOF1 (PDO) and EOF2 (NPGO) to project the monthly SSTa to reconstruct monthly indices for both PDO and NPGO.

### Why do it this way?

The expressions of PDO and NPGO are well defined during JFM in this region. This is because the atmospheric forcing is strongest during JFM. 

### References

1. Di Lorenzo, E. and N. Mantua, 2016: Multi-year persistence of the 2014/15 North Pacific marine heatwave. Nature Climate Change, 6(11) 1042-+, doi:10.1038/nclimate3082. (**Especially the supplemental material**)

2. Joh, Y. and E. Di Lorenzo: Increasing coupling between NPGO and PDO leads to prolonged marine heatwaves in the Northeast Pacific. Geophysical Research Letters, in review.

In [75]:
import numpy as np
import pandas as pd
import xarray as xr
import esmtools as et
from eofs.xarray import Eof
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
%matplotlib inline
plt.style.use('ocn-clim')
import glob

In [2]:
# Load in regridded SST residuals (global) from one simulation.
ds = xr.open_dataset('/glade/scratch/rbrady/EBUS_BGC_Variability/global_residuals/SST/' +
                     'remapped/remapped.SST.009.192001-201512.nc')
ds = ds['SST']
print(ds)

<xarray.DataArray 'SST' (time: 1152, lat: 180, lon: 360)>
[74649600 values with dtype=float64]
Coordinates:
  * lon      (lon) float64 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 11.0 ...
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 -84.5 -83.5 -82.5 ...
  * time     (time) datetime64[ns] 1920-01-31 1920-02-29 1920-03-31 ...


In [3]:
# Slice down to the Northeast Pacific domain.
ds = ds.sel(lat=slice(25, 62), lon=slice(180, 250))

# Winter Averages

In [4]:
# Take annual JFM means.
month = ds['time.month']
JFM = (month <= 3)
ds_winter = ds.where(JFM).resample('A','time')

In [5]:
# Compute EOF.
coslat = np.cos(np.deg2rad(ds_winter.lat.values))
wgts = np.sqrt(coslat)[..., np.newaxis]
# Center is false because I already computed the
# anomalies by removing the ensemble mean.
solver = Eof(ds_winter, weights=wgts, center=False)

In [7]:
eof = solver.eofsAsCorrelation(neofs=2)
#pc = solver.pcs(npcs=2, pcscaling=1)
variance = solver.varianceFraction(neigs=2)
# Reconstruct monthly index
pseudo_pcs = solver.projectField(ds, neofs=2, eofscaling=1)

/glade/u/home/rbrady/.local/lib/python3.6/site-packages/eofs/xarray.py:271: FutureWarning: Coordinate has been deprecated. Use IndexVariable instead.
  attrs={'long_name': 'eof_mode_number'})
/glade/u/home/rbrady/.local/lib/python3.6/site-packages/eofs/xarray.py:413: FutureWarning: Coordinate has been deprecated. Use IndexVariable instead.
  attrs={'long_name': 'eof_mode_number'})
/glade/u/home/rbrady/.local/lib/python3.6/site-packages/eofs/xarray.py:625: FutureWarning: Coordinate has been deprecated. Use IndexVariable instead.
  attrs={'long_name': 'eof_mode_number'})


In [ ]:
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
def EOF_map():
    fig, ax = plt.subplots(figsize=(8,6), subplot_kw=dict(projection=ccrs.PlateCarree()))
    gl = ax.gridlines(draw_labels=True, color='w', alpha=0.5)
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.add_feature(cfeature.LAND, facecolor='k')
    ax.outline_patch.set_edgecolor('white')
    return fig, ax

In [ ]:

f, ax = EOF_map()
p = eof[0].plot.contourf(transform=ccrs.PlateCarree(), add_colorbar=False,
                cmap="RdYlBu_r",
                levels=np.arange(-0.8, 0.81, 0.1))
ax.set_extent([-180,-115,25,62])
plt.colorbar(p, orientation='horizontal', fraction=0.05, pad=0.05)
ax.set_title('EOF 1 of JFM SSTa' + '\n' + '[PDO]')

In [ ]:
f, ax = EOF_map()
p = eof[1].plot.contourf(transform=ccrs.PlateCarree(), add_colorbar=False,
                cmap="RdYlBu_r",
                levels=np.arange(-0.8, 0.81, 0.1))
ax.set_extent([-180,-115,25,62])
plt.colorbar(p, orientation='horizontal', fraction=0.05, pad=0.05)
ax.set_title('EOF 2 of JFM SSTa' + '\n' + '[NPGO]')

# Reconstructed monthly indices

In [ ]:
f = plt.figure(figsize=(14,3))
ax = f.add_subplot(111)
pseudo_pcs[:,0].plot(linewidth=2, color='r', label='Brady')
ax.set(xlim=['1920','2015-12'], title='PC 1', ylabel='')
ax.legend()

# Bring in Adam Phillip's PDO

In [ ]:
ds_clim = xr.open_dataset('/glade/p/work/rbrady/cesmLE_CVDP/CESM1-CAM5-BGC-LE_009.cvdp_data.1920-2015.nc',
                          decode_times=False)

In [ ]:
ds_clim = ds_clim['pdo_timeseries_mon']

In [ ]:
ds_clim['time'] = pd.date_range('1920-01','2015-12',freq='M')

In [ ]:
ds_clim['time'] = pseudo_pcs['time']

In [ ]:
f = plt.figure(figsize=(14,3))
ax = f.add_subplot(111)
pseudo_pcs[:,0].plot(linewidth=2, color='r', label='Brady')
ds_clim.plot(linewidth=2, color='k', label='Phillips')
ax.set(xlim=['1920','2015-12'], title='PC 1', ylabel='')
ax.legend()